In [1]:

# Set up the notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

# Load libraries
from FRVRS import nu, fu
import os.path as osp
from pandas import DataFrame
import pandas as pd


# Data Fixes for Metrics Evaluation Open World

In [3]:

# load data frames
data_frames_dict = nu.load_data_frames(
    metrics_evaluation_open_world_df='', metrics_evaluation_open_world_json_stats_df='', metrics_evaluation_open_world_scene_stats_df=''
)
metrics_evaluation_open_world_df = data_frames_dict['metrics_evaluation_open_world_df']
metrics_evaluation_open_world_json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']
metrics_evaluation_open_world_scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df']

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl.


In [4]:

scene_columns_set = set(metrics_evaluation_open_world_scene_stats_df.columns)
logs_columns_set = set(metrics_evaluation_open_world_df.columns)
intersection_columns = set(['is_scene_aborted'])

# Drop the logs columns already recorded in the scene stats data frames
drop_columns = sorted(scene_columns_set.intersection(logs_columns_set).intersection(intersection_columns))
print(drop_columns)
if drop_columns:
    metrics_evaluation_open_world_df = metrics_evaluation_open_world_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_df.shape) # (171766, 107)
    nu.store_objects(metrics_evaluation_open_world_df=metrics_evaluation_open_world_df)
    nu.save_data_frames(metrics_evaluation_open_world_df=metrics_evaluation_open_world_df)

[]


In [5]:

logs_columns_set = set(metrics_evaluation_open_world_df.columns)
file_columns_set = set(metrics_evaluation_open_world_json_stats_df.columns)
intersection_columns = set(['logger_version', 'is_scene_aborted'])

# Drop the logs columns already recorded in the JSON and scene stats data frames
drop_columns = list(logs_columns_set.intersection(file_columns_set).intersection(intersection_columns))
print(drop_columns)
if drop_columns:
    metrics_evaluation_open_world_df = metrics_evaluation_open_world_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_df.shape) # (171766, 124)
    nu.store_objects(metrics_evaluation_open_world_df=metrics_evaluation_open_world_df)
    nu.save_data_frames(metrics_evaluation_open_world_df=metrics_evaluation_open_world_df)

[]


In [6]:

logs_columns_set = set(metrics_evaluation_open_world_df.columns)
file_columns_set = set(metrics_evaluation_open_world_json_stats_df.columns)
intersection_columns = set([
    'injury_record_injury_treated', 'injury_record_injury_treated_with_wrong_treatment', 'patient_demoted_health_level', 'patient_demoted_health_time_remaining',
    'patient_demoted_hearing', 'patient_hearing', 'patient_record_health_level', 'patient_record_hearing', 'player_location_left_hand_location',
    'player_location_right_hand_location', 'bag_access_location', 'patient_engaged_health_level', 'voice_capture_command_description'
])

# Drop the JSON Stats columns that came with the process but are covered well enough in the logs data frame and add no value here
drop_columns = sorted(logs_columns_set.intersection(file_columns_set).intersection(intersection_columns))
print(drop_columns)
if drop_columns:
    metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3589)
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)

['bag_access_location', 'injury_record_injury_treated', 'injury_record_injury_treated_with_wrong_treatment', 'patient_demoted_health_level', 'patient_demoted_health_time_remaining', 'patient_engaged_health_level', 'patient_record_health_level', 'player_location_left_hand_location', 'player_location_right_hand_location', 'voice_capture_command_description']
(43, 3561)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv


In [7]:

file_columns_set = set(metrics_evaluation_open_world_json_stats_df.columns)
scene_columns_set = set(metrics_evaluation_open_world_scene_stats_df.columns)
intersection_columns = set(['logger_version'])

# Drop the scene columns already recorded in the JSON Stats data frames
drop_columns = sorted(file_columns_set.intersection(scene_columns_set).intersection(intersection_columns))
if drop_columns:
    metrics_evaluation_open_world_scene_stats_df = metrics_evaluation_open_world_scene_stats_df.drop(columns=drop_columns)
    print(metrics_evaluation_open_world_scene_stats_df.shape) # (76, 48)
    nu.store_objects(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)

In [8]:

# Remove duplicates from the JSON Stats data frame
subset_columns = ['session_uuid']
mask_series = metrics_evaluation_open_world_json_stats_df.duplicated(subset=subset_columns)
if mask_series.any():
    metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df[~mask_series]
    print(metrics_evaluation_open_world_json_stats_df.shape)
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)

In [9]:

# Remove duplicates from the scene stats data frame
subset_columns = ['session_uuid', 'scene_id']
mask_series = metrics_evaluation_open_world_scene_stats_df.duplicated(subset=subset_columns)
if mask_series.any():
    metrics_evaluation_open_world_scene_stats_df = metrics_evaluation_open_world_scene_stats_df[~mask_series]
    print(metrics_evaluation_open_world_scene_stats_df.shape)
    nu.store_objects(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_scene_stats_df=metrics_evaluation_open_world_scene_stats_df)


### Get column and value descriptions

In [10]:

# Attempt to manufacture some better column names
from pandas import read_excel

file_path = '../data/xlsx/Metrics_Evaluation_Dataset_organization_for_BBAI.xlsx'
dataset_organization_df = read_excel(file_path)

mask_series = ~dataset_organization_df.Description.isnull()#re.sub('[^A-Za-z0-9]+', '_', x)
df = dataset_organization_df[mask_series]
description_dict = df.set_index('Variable').Description.to_dict()
new_description_dict = description_dict.copy()
for k, v in description_dict.items():
    new_description_dict[k] = v
    new_description_dict[f'{k}_Text'] = v
description_dict = new_description_dict.copy()

In [11]:

# Fix the doubled up descriptions
import re

mask_series = dataset_organization_df.Labels.map(lambda x: ';' in str(x))
for row_index, label in dataset_organization_df[mask_series].Labels.items():
    labels_list = re.split(' *; *', str(label), 0)
    dataset_organization_df.loc[row_index, 'Labels'] = labels_list[0]
    
    # Get a copy of the row
    new_row = dataset_organization_df.loc[row_index].copy()
    
    # Modify the desired column value
    new_row['Labels'] = labels_list[1]
    
    # Append the new row to the DataFrame
    dataset_organization_df = dataset_organization_df.append(new_row, ignore_index=True)

In [12]:

# Get a copy of the row
mask_series = (dataset_organization_df.Variable == 'AD_Del_Omni')
new_row = dataset_organization_df.loc[mask_series].copy()

# Modify the desired column value
new_row['Variable'] = 'AD_Del_Omni_Text'

# Append the new row to the DataFrame
dataset_organization_df = dataset_organization_df.append(new_row, ignore_index=True)

In [13]:

from numpy import nan

mask_series = dataset_organization_df.Labels.map(lambda x: '=' in str(x))
value_descriptions_columns = dataset_organization_df[mask_series].Variable.unique().tolist()
def get_value_description(column_name, column_value):
    value_description = ''
    if not pd.isna(column_value):
        mask_series = (dataset_organization_df.Variable == column_name) & ~dataset_organization_df.Labels.isnull()
        if mask_series.any():
            df = dataset_organization_df[mask_series]
            mask_series = df.Labels.map(lambda label: re.split(' *= *', str(label), 0)[0] == str(int(float(column_value))))
            if mask_series.any():
                label = df[mask_series].Labels.squeeze()
                value_description = re.split(' *= *', str(label), 0)[1]
    
    return value_description
column_name = 'MedRole'
column_value = nan
get_value_description(column_name, column_value)

''


## Provide Correctly Grouped Responder Type Stats

In [14]:

data_frames_dict = nu.load_data_frames(first_responder_master_registry_file_stats_df='')
df = data_frames_dict['first_responder_master_registry_file_stats_df']
mask_series = ~df.responder_category.isnull()
sorted(df[mask_series].responder_category.unique())

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_file_stats_df.pkl.


['EM Resident', 'EMT-Basic', 'Medical Student', 'Other', 'Paramedic']

In [15]:

[cn for cn in metrics_evaluation_open_world_json_stats_df.columns if 'Med' in cn]

['configData_narrative_narrativeSections00_availableInventory_painMedsCount', 'configData_narrative_narrativeSections01_availableInventory_painMedsCount', 'configData_narrative_narrativeSections02_availableInventory_painMedsCount', 'configData_narrative_narrativeSections03_availableInventory_painMedsCount', 'configData_narrative_narrativeSections04_availableInventory_painMedsCount', 'configData_narrative_narrativeSections05_availableInventory_painMedsCount', 'configData_narrative_narrativeSections06_availableInventory_painMedsCount', 'configData_narrative_narrativeSections07_availableInventory_painMedsCount', 'configData_narrative_narrativeSections08_availableInventory_painMedsCount', 'configData_narrative_narrativeSections09_availableInventory_painMedsCount', 'configData_narrative_narrativeSections10_availableInventory_painMedsCount', 'configData_narrative_narrativeSections11_availableInventory_painMedsCount', 'configData_narrative_narrativeSections12_availableInventory_painMedsCount'

In [18]:

# Add a column that correctly groups responder types
new_column_name = 'responder_category'
if (new_column_name in metrics_evaluation_open_world_json_stats_df.columns):
    metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df.drop(columns=[new_column_name])
    print(metrics_evaluation_open_world_json_stats_df.shape)
if (new_column_name not in metrics_evaluation_open_world_json_stats_df.columns):
    metrics_evaluation_open_world_json_stats_df[new_column_name] = metrics_evaluation_open_world_json_stats_df.MedRole.map(
        lambda x: ' '.join([r.title() for r in get_value_description('MedRole', x).split(' ')]).replace('Em ', 'EM ')
    )
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3564)

display(metrics_evaluation_open_world_json_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

(43, 3561)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv
(43, 3562)


,record_count
responder_category,
Medical Student,17
Other,16
,4
EM Faculty,2
EM Resident,2


In [19]:

# Add responder types subgrouping columns
groupby_columns = ['overall_category', 'global_category', 'global_description', 'sub_category', 'sub_description', 'responder_type']
if any(map(lambda x: x not in metrics_evaluation_open_world_json_stats_df.columns, groupby_columns)):
    file_path = '../data/xlsx/Responder_Categories_and_Counts_DPW.xlsx'
    dpw_responder_categories_df = pd.read_excel(file_path)

    # Get description data frame
    mask_series = dpw_responder_categories_df.isna().all(axis='columns')
    idx = dpw_responder_categories_df[mask_series].index.min()
    where_df = dpw_responder_categories_df.iloc[idx+1:]
    
    # Get categories data frame
    dpw_responder_categories_df = dpw_responder_categories_df.iloc[:idx].dropna(axis='columns', how='all')
    dpw_responder_categories_df.columns = ['overall_category', 'responder_type', 'global_category', 'sub_category', 'record_count']
    
    # Create global description column
    columns_list = ['global_category', 'global_description', 'record_count']
    df = where_df.iloc[:, 1:4].dropna(axis='index', how='all')
    df.columns = columns_list
    global_description_dict = df.set_index('global_category').global_description.to_dict()
    dpw_responder_categories_df['global_description'] = dpw_responder_categories_df.global_category.map(global_description_dict)
    
    # Create sub description column
    columns_list = ['sub_category', 'sub_description', 'record_count']
    df = where_df.iloc[:, 5:8].dropna(axis='index', how='all')
    df.columns = columns_list
    sub_description_dict = df.set_index('sub_category').sub_description.to_dict()
    dpw_responder_categories_df['sub_description'] = dpw_responder_categories_df.sub_category.map(sub_description_dict)
    
    # Add columns to JSON Stats data frame
    df = dpw_responder_categories_df[groupby_columns].groupby(groupby_columns).size().reset_index(drop=False)
    assert not (df[0] != 1).any(), "You will have a problem with responder types"
    df = df.drop(columns=[0])
    assert not (df.groupby('responder_type').size() != 1).any(), "You have a problem with responder types"
    if 'responder_type' not in metrics_evaluation_open_world_json_stats_df.columns: metrics_evaluation_open_world_json_stats_df['responder_type'] = 'EM-RES1'
    for (overall_category, global_category, global_description, sub_category, sub_description, responder_type), _ in df.groupby(groupby_columns):
        mask_series = (metrics_evaluation_open_world_json_stats_df.responder_type == responder_type)
        for cn in groupby_columns: metrics_evaluation_open_world_json_stats_df.loc[mask_series, cn] = eval(cn)
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3570)

if all(map(lambda x: x in metrics_evaluation_open_world_json_stats_df.columns, groupby_columns)):
    print(metrics_evaluation_open_world_json_stats_df[groupby_columns].nunique()) 
    display(metrics_evaluation_open_world_json_stats_df.groupby(groupby_columns).size().to_frame().rename(columns={0: 'record_count'}).sort_values(
        'record_count', ascending=False
    ).head(5))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv
(43, 3568)
overall_category      1
global_category       1
global_description    1
sub_category          1
sub_description       1
responder_type        1
dtype: int64


,,,,,,record_count
overall_category,global_category,global_description,sub_category,sub_description,responder_type,
EM Resident,DOC,Physicians,DRS,Physician Resident,EM-RES1,43


In [ ]:

# Try to base the CACI data on the sessions in their spreadsheet
from pandas import read_excel

data_frames_dict = nu.load_data_frames(
    metrics_evaluation_open_world_df='', metrics_evaluation_open_world_json_stats_df='', metrics_evaluation_open_world_scene_stats_df=''
)
logs_df = data_frames_dict['metrics_evaluation_open_world_df']
json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']
scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df']
print(logs_df.shape)
file_path = '../data/xlsx/Metrics_Eval_Participant_data_for_BBAI.xlsx'
participant_data_df = read_excel(file_path)
uuid_columns = [f'Sim{i}' for i in range(1, 3)]
uuid_fn = lambda x: str(x)[:-1] if str(x).endwith('_') else x
json_stats_uuids = []
for participant_id in range(2_024_201, 2_024_223+1):
    mask_series = (participant_data_df.ParticipantID == participant_id)
    if mask_series.any():
        participant_df = participant_data_df[mask_series]
        on_columns = set(json_stats_df.columns).intersection(set(participant_df.columns))
        session_uuids_list = participant_df[uuid_columns].applymap(uuid_fn).values.ravel().tolist()
        mask_series = json_stats_df.session_uuid.isin(session_uuids_list)
        if mask_series.any(): json_stats_uuids.extend(json_stats_df[mask_series].session_uuid.tolist())
    else: print(f'You are missing Participant ID #{participant_id} from the Metrics Eval Participant data for BBAI')
print(logs_df.shape)
# nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df, verbose=True)
# nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df, verbose=True)

In [16]:

import datetime

# Filter rows with non-null 'Date' values
mask_series = ~json_stats_df.Date.isnull()
filtered_df = json_stats_df[mask_series]

# Get minimum and maximum dates as datetime objects
min_date = filtered_df['Date'].min()
max_date = filtered_df['Date'].max()

# Format the datetime objects for human-readable output
min_date_str = datetime.datetime.strptime(min_date, '%m/%d/%Y').strftime('%B %d, %Y')
max_date_str = datetime.datetime.strptime(max_date, '%m/%d/%Y').strftime('%B %d, %Y')

# Print the results
print(
    f'A total of 22 participants, representing diverse medical roles, completed the ITM scenarios between {min_date_str} and {max_date_str}.'
    ' Each participant engaged with two separate open world environments.'
)

A total of 22 participants, representing diverse medical roles, completed the ITM scenarios between March 14, 2024 and March 22, 2024. Each participant engaged with two separate open world environments.


In [20]:

# Get the open world action types
import re
import os

action_type_regex = re.compile("^([^,\r\n]+),", re.MULTILINE)
file_ending = '.csv'
logs_path = '../data/logs/Metrics Evaluation Open World'
all_actions_set = set()
for sub_directory, directories_list, files_list in os.walk(logs_path):
    for file_name in files_list:
        if file_name.endswith(file_ending):
            file_path = osp.join(sub_directory, file_name)
            with open(file_path, 'r', encoding=nu.encoding_type) as f:
                text = f.read()
                actions_set = set(action_type_regex.findall(text))
                all_actions_set.update(actions_set)

old_actions_list = [
    'BAG_ACCESS', 'BAG_CLOSED', 'INJURY_RECORD', 'INJURY_TREATED', 'PATIENT_DEMOTED', 'PATIENT_ENGAGED', 'PATIENT_RECORD', 'PULSE_TAKEN',
    'S_A_L_T_WALKED', 'S_A_L_T_WALK_IF_CAN', 'S_A_L_T_WAVED', 'S_A_L_T_WAVE_IF_CAN', 'TAG_APPLIED', 'TAG_DISCARDED', 'TAG_SELECTED', 'TELEPORT',
    'TOOL_APPLIED', 'TOOL_DISCARDED', 'TOOL_HOVER', 'TOOL_SELECTED', 'VOICE_CAPTURE', 'VOICE_COMMAND', 'PLAYER_LOCATION', 'PLAYER_GAZE',
    'SESSION_END', 'SESSION_START'
]
new_actions_list = sorted(all_actions_set.difference(set(old_actions_list)))
print(new_actions_list)

['BREATHING_CHECKED', 'BUTTON_CLICKED', 'Participant ID', 'SP_O2_TAKEN', 'TRIAGE_LEVEL_WALKED', 'TRIAGE_LEVEL_WALK_IF_CAN', 'TRIAGE_LEVEL_WAVED', 'TRIAGE_LEVEL_WAVE_IF_CAN']
